In [8]:
import timesfm
import pandas as pd

In [51]:
tfm = timesfm.TimesFm(
      hparams=timesfm.TimesFmHparams(
          backend="gpu",
          per_core_batch_size=32,
          horizon_len=10,
          num_layers=50,
          use_positional_embedding=False,
          context_len=2048,
      ),
      checkpoint=timesfm.TimesFmCheckpoint(
          huggingface_repo_id="google/timesfm-2.0-500m-pytorch"),
  )

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 48545.19it/s]


Create a function to process a dataframe

In [52]:
def process_dataframe(df):
    df = df.rename(
        columns={
            "date": "ds",
            "price": "y"
        }
    )
    df['ds'] = pd.to_datetime(df['ds'])
    df['unique_id'] = "bitcoin"
    return df

In [53]:
df = pd.read_csv(
    "/Users/sebastianalejandrosarastizambonino/Documents/conferences/time_series_u_palermo/data/bitcoin_history_yf.csv",
)

In [54]:
df_final = process_dataframe(df)

Create a function to make inference over the model

In [55]:
help(tfm.forecast_on_df)

Help on method forecast_on_df in module timesfm.timesfm_base:

forecast_on_df(inputs: pandas.core.frame.DataFrame, freq: str, forecast_context_len: int = 0, value_name: str = 'values', model_name: str = 'timesfm', window_size: int | None = None, num_jobs: int = 1, normalize: bool = False, verbose: bool = True) -> pandas.core.frame.DataFrame method of timesfm.timesfm_torch.TimesFmTorch instance
    Forecasts on a list of time series.
    
    Args:
      inputs: A pd.DataFrame of all time series. The dataframe should have a
        `unique_id` column for identifying the time series, a `ds` column for
        timestamps and a value column for the time series values.
      freq: string valued `freq` of data. Notice this is different from the
        `freq` required by `forecast`. See `freq_map` for allowed values.
      forecast_context_len: If provided none zero, we take the last
        `forecast_context_len` time-points from each series as the forecast
        context instead of the `c

In [56]:
def predict_timesfm(df, model):
    forecast_df = model.forecast_on_df(
        inputs=df,
        forecast_context_len=10,
        freq="D",  # monthly
        value_name="y",
        num_jobs=-1,
    )
    forecast_df = forecast_df[['ds', 'unique_id', 'timesfm']]
    forecast_df = forecast_df.rename(
        columns={
            "timesfm": "yhat"
        }
    )
    return forecast_df

In [57]:
df_final.columns

Index(['ds', 'y', 'unique_id'], dtype='object')

In [58]:
forecast = predict_timesfm(df_final)

Processing dataframe with multiple processes.
 See https://github.com/google-research/timesfm/blob/master/README.md for updated APIs.
Loaded PyTorch TimesFM, likely because python version is 3.11.13 (main, Jun  5 2025, 08:21:08) [Clang 14.0.6 ].
Finished preprocessing dataframe.
Finished forecasting.


In [59]:
forecast

,ds,unique_id,yhat
0,2025-01-01 00:00:00+00:00,bitcoin,93821.898438
1,2025-01-02 00:00:00+00:00,bitcoin,93758.367188
2,2025-01-03 00:00:00+00:00,bitcoin,93707.375000
3,2025-01-04 00:00:00+00:00,bitcoin,93779.257812
4,2025-01-05 00:00:00+00:00,bitcoin,93857.195312
5,2025-01-06 00:00:00+00:00,bitcoin,93959.531250
6,2025-01-07 00:00:00+00:00,bitcoin,94230.304688
7,2025-01-08 00:00:00+00:00,bitcoin,94447.601562
8,2025-01-09 00:00:00+00:00,bitcoin,94440.648438
9,2025-01-10 00:00:00+00:00,bitcoin,94379.914062
